<a href="https://colab.research.google.com/github/Baudier13/RN2022/blob/main/entrenamiento_validacion_prueba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conjuntos de datos de entrenamiento, validación y prueba

**Refs**

https://machinelearningmastery.com/difference-test-validation-datasets/

https://www.geeksforgeeks.org/training-neural-networks-with-validation-using-pytorch/

https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import sklearn as skl
import pandas as pd
#from torchviz import make_dot
import torch.optim as optim
from collections import defaultdict
import pickle
import dill
import json
import datetime
try:
  import google.colab
  from google.colab import files  
  COLAB = True
except:
  COLAB = False

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Usando el dispositivo {}'.format(device))

Consideremos una familia de redes neuronales $y = f_h(x;w_h)$ indexada por la elección de conjunto de hiperparámetros $h$, ej. la arquitectura de la red, el algoritmo de entrenamiento, el número de épocas de entrenamiento, etc.
Aquí, $x$ denota la entrada (ej. features) de la red, $y$ la salida (ej. labels) y $w_h$ los parámetros o pesos sinápticos de la misma.

Distintas elecciones de los hiperparámetros pueden ser convenientes para aprender datasets de distintas características y/o complejidades.

Sabemos que redes demasiado simples (con pocos parámetros) no logran aprender datasets suficientemente complejos, y que redes demasiado complejas tienden a sobrefitear datos.
Por ende, nos interesa elegir una red de la familia que sea capáz de aprender los datos a disposición y que presente buenas características de generalización.
Para ello, dividimos el conjunto de datos a disposición (el cuál se supone estar compuesto de muestras generadas de manera estadísticamente independientes) en tres conjuntos:

1. el conjunto de entrenamiento (training),

2. el conjunto de validación (validation), y

3. el conjunto de prueba (test).

Luego, buscando optimizar sobre la elección de hiperparámetros, realizamos el siguiente procedimiento para cada valor de $h$:

1. Entrenamos la red $f_h(x,w_h)$ optimizando con respecto a $w_h$ sobre las muestras $x$ obtenidas de dataset de entrenamiento, usando una métrica de nuestra preferencia; ej. la *loss* (pérdida) o la *precission* (precisión).
Esto resulta en valores "optimos" de los parámetros $\hat{w}_h$, de manera que $f_h(x,\hat{w}_h)$ constituye una red entrenada.

2. Luego, usando la misma métrica, evaluamos $f_h(x,\hat{w}_h)$ sobre el conjunto de validación, para ver cuán bien generaliza la red ya entrenada sobre datos que no fueron utilizados durante la etapa de entrenamiento (i.e de optimización de $w_h$).

Luego, elegimos la arquitectura $\hat{h}$ que haya dado los mejores resultados durante el paso de validación 2, caracterizando las bondades de nuestra elección $f_{\hat{h}}(x;\hat{w}_{\hat{h}})$ evaluándola sobre el conjunto de prueba (test) que no ha sido utilizado ni durante el proceso de entrenamiento, ni durante el proceso de evaluación.

Veamos un ejemplo con **FashionMNIST** y una red multicapa de sólo una capa oculta.
Para ello, comenzamos por crear los conjuntos de entrenamiento, validación y testeo.

In [ ]:
# La primera vez esto tarda un rato ya que tiene que bajar los datos de la red.
labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}
train_dataset = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
test_dataset = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

Guardamos el train_dataset original para luego poder divirlo en subconjuntos de validación y prueba.

In [ ]:
train_dataset_orig = train_dataset
len(train_dataset_orig)

Luego definimos la red neuronal.
Esta es un perceptron con una capa oculta de tamaño arbitrario $n$. 
En este ejemplo, dicho $n$ es el único grado de libertad que dejamos variar, de entre todos los que definen la arquitectura de la red.
En otras palabras, nuestra familia estará compuesta de redes con capas ocultas de distintos tamaños.

In [ ]:
class Net(nn.Module):
    def __init__(self,n=128):
        super(Net,self).__init__()
        self.flatten = nn.Flatten()
        self.relu = nn.ReLU()
        self.linear1 = nn.Linear(28*28,n)
        self.linear2 = nn.Linear(n,10)
    def forward(self,x):
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        return x

Implementamos las funciones para entrenar, validar y testear un modelo.

In [ ]:
# Definimos la función de entrenamiento
def train_loop(dataloader,model,loss_fn,optimizer,verbose_each=32):  
    # Calculamos cosas utiles que necesitamos
    num_samples = len(dataloader.dataset)
    # Seteamos el modelo en modo entrenamiento. Esto sirve para activar, por ejemplo, dropout, etc. durante la fase de entrenamiento.
    model.train()
    # Pasamos el modelo la GPU si está disponible.        
    model = model.to(device)    
    # Iteramos sobre lotes (batchs)
    for batch,(X,y) in enumerate(dataloader):
        # Pasamos los tensores a la GPU si está disponible.
        X = X.to(device)
        y = y.to(device)      
        # Calculamos la predicción del modelo y la correspondiente pérdida (error)
        pred = model(X)
        loss = loss_fn(pred,y)
        # Backpropagamos usando el optimizador proveido.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Imprimimos el progreso cada 100 batchs
        if batch % verbose_each*len(X) == 0:
            loss   = loss.item()
            sample = batch*len(X) # Número de batch * número de muestras en cada batch
            #print(f"batch={batch} loss={loss:>7f}  muestras-procesadas:[{sample:>5d}/{num_samples:>5d}]")            
# De manera similar, definimos la función de validación y testeo
def test_loop(dataloader,model,loss_fn):
    num_samples  = 0
    num_batches  = 0
    avrg_loss    = 0
    frac_correct = 0
    # Seteamos el modelo en modo evaluacion. Esto sirve para desactivar, por ejemplo, dropout, etc. cuando no estamos en una fase de entrenamiento.
    model.eval()
    # Pasamos el modelo la GPU si está disponible.    
    model = model.to(device)    
    # Para validar, desactivamos el cálculo de gradientes.
    with torch.no_grad():
        # Iteramos sobre lotes (batches)
        for X,y in dataloader:
            # Pasamos los tensores a la GPU si está disponible.
            X = X.to(device)
            y = y.to(device)           
            # Calculamos las predicciones del modelo...
            pred = model(X)
            # y las correspondientes pérdidas (errores), los cuales vamos acumulando en un valor total.
            num_batches += 1
            avrg_loss += loss_fn(pred,y).item()
            # También calculamos el número de predicciones correctas, y lo acumulamos en un total.
            num_samples += y.size(0)            
            frac_correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    # Calculamos la pérdida total y la fracción de clasificaciones correctas, y las imprimimos.
    avrg_loss    /= num_batches
    frac_correct /= num_samples
    #print(f"Test Error: \n Accuracy: {frac_correct:>0.5f}, Avg. loss: {avrg_loss:>8f} \n")
    return avrg_loss,frac_correct

Implementamos un simple método de validación cruzada para explorar que valores de hiperparámetros (en este caso, el tamaño $n$ de la capa oculta y el número $\mathsf{epoch}$ óptimo de épocas de entrenamiento) conviene seleccionar.
El método de validación cruzada consiste en dividir los datos de entrenamiento (60 muestras) en dos subconjuntos, uno de entrenamiento más pequeño (50 muestras) y otro de validación (10 muestras). 
Estos subconjuntos se generan en cada iteración del proceso de validación tras aleatorizar el orden de las muestras. 

In [ ]:
# Definimos hiperparámetros de entrenamiento
learning_rate = 1e-3
batch_size = 500
num_epochs = 100
num_k = 1 #72
n=2048 # Recordar que 28*28=784
# Creamos una funcion de perdida
loss_fn = nn.CrossEntropyLoss()
# Creamos un DataFrame de pandas para ir almacenando los valores calculados.
df = pd.DataFrame()
# Simulamos por tramos porque google colab se desconecta antes de que concluya para todos los valores de n en la lista.
for k in range(num_k):
    # Creamos el modelo y el optimzador
    model = Net(n)
    # Creamos los dataloaders ...
    train_dataloader = DataLoader(train_dataset,batch_size=batch_size)
    # ... en particular, usamos el dataset de prueba (test) como dataset de validación
    valid_dataloader = DataLoader(test_dataset,batch_size=batch_size)         
    #optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
    optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate,eps=1e-08,weight_decay=0,amsgrad=False)
    # Entrenamos el modelo y calcualmos curvas.
    min_valid_loss = float("inf")
    for epoch in range(num_epochs):
        train_loop(train_dataloader,model,loss_fn,optimizer)
        train_loss,train_accu = test_loop(train_dataloader,model,loss_fn)
        valid_loss,valid_accu = test_loop(valid_dataloader,model,loss_fn)
        print(f"n={n} k={k} epoch={epoch} train_loss={train_loss} train_accu={train_accu} valid_loss={valid_loss} valid_accu={valid_accu}")
        df = df.append({"n":n,
                        "k":k,
                        "epoch":epoch,
                        "train_loss":train_loss,
                        "train_accu":train_accu,
                        "valid_loss":valid_loss,
                        "valid_accu":valid_accu}
                        ,ignore_index=True)
json_fname = "simulation-results-"+datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")+".json"
df.to_json(json_fname)
if COLAB:
    files.download(json_fname)

**Simulation summary**

simulation-results-2021-12-15-18-37-04.json

simulation-results-2021-12-15-18-59-55.json

simulation-results-2021-12-15-19-18-31.json

simulation-results-2021-12-15-19-42-48.json

simulation-results-2021-12-15-20-39-16.json

simulation-results-2021-12-15-21-10-38.json

simulation-results-2021-12-15-22-38-52.json

simulation-results-2021-12-15-23-46-54.json

In [ ]:
#df = pd.read_json(json_fname)
#df

In [ ]:
%%bash --out list_json
# Usamos el bash magic de Jupyter para ver que archivos *.json hemos creado.
# Guardamos el resultado en la variable list_json
ls *.json

In [ ]:
list_json = list_json.split()
list_json

In [ ]:
df = pd.concat([pd.read_json(json_fname) for json_fname in list_json],ignore_index=True)
df

In [ ]:
df1 = df.drop("k",1)
df1

In [ ]:
df2 = df1.pivot_table(index=["n","epoch"],aggfunc="count").reset_index()
df2

In [ ]:
df3 = df1.pivot_table(index=["n","epoch"],aggfunc="mean").reset_index()
df3

Visualicemos el desempeño de cada arquitectura de red

In [ ]:
fig,axes=plt.subplots(1,2)
fig.set_size_inches(10.0,5.0)
colors = cm.Dark2.colors
for color,n in zip(colors,df["n"].unique()):
    dfn = df3[df3["n"]==n]
    x = dfn["epoch"]
    ax = axes[0]
    ax.set_xlabel("epoch")
    ax.set_ylabel("loss")
    ax.plot(x,dfn["train_loss"],label=f"train n={n}",color=color)
    ax.plot(x,dfn["valid_loss"],label=f"valid n={n}",color=color,linestyle='--')
    ax.legend()
    ax = axes[1]
    ax.set_xlabel("epoch")
    ax.set_ylabel("accuracy")
    ax.plot(x,dfn["train_accu"],label=f"train n={n}",color=color)
    ax.plot(x,dfn["valid_accu"],label=f"test n={n}",color=color,linestyle='--')
    ax.legend()
fig.tight_layout()
plt.show()    

In [ ]:
df4 = df3.pivot_table(index=["n"],
                    aggfunc={
                        "train_loss":min,
                        "valid_loss":min,
                        "train_accu":max,
                        "valid_accu":max,
                    }
                   ).reset_index()
df4

In [ ]:
x=df4["n"]
fig,axes=plt.subplots(1,2)
fig.set_size_inches(10.0,5.0)
ax = axes[0]
ax.set_xlabel("n")
ax.set_ylabel("min. loss")
ax.scatter(x,df4["train_loss"],label=f"train")
ax.plot(x,df4["train_loss"],label=f"train")
ax.scatter(x,df4["valid_loss"],label=f"valid",linestyle='--')
ax.plot(x,df4["valid_loss"],label=f"valid",linestyle='--')
ax.set_xscale("log")
#ax.set_yscale("log")
ax.legend()
ax = axes[1]
ax.set_xlabel("n")
ax.set_ylabel("max. accuracy")
ax.scatter(x,df4["train_accu"],label=f"train")
ax.plot(x,df4["train_accu"],label=f"train")
ax.scatter(x,df4["valid_accu"],label=f"valid",linestyle='--')
ax.plot(x,df4["valid_accu"],label=f"valid",linestyle='--')
ax.set_xscale("log")
ax.legend()
fig.tight_layout()
plt.show()    

La tendencia general es que la loss de validación decrece con $n$ y la accuracy de validación crece con $n$.
Siendo un poco más detallistas, pero despreciando fluctuaciones, podríamos decir que el crecimiento de la accuracy de validación se estanca a partir de $n=512$.
Por ende, tomamos dicho tamaño como el óptimo.

Reentrenamos el modelo para el caso óptimo de $n$ anteriormente determinado, y optimizando sobre $\mathsf{epoch}$ utilizando un símple método de validación cruzada, para luego evaluarlo en el conjunto de prueba.

In [ ]:
# Definimos hiperparámetros de entrenamiento
init_datetime = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
learning_rate = 1e-3
batch_size = 1000
num_epochs = 40
num_k = 12 #72
# Recordar que 28*28=784
n=512
# Creamos una funcion de perdida
loss_fn = nn.CrossEntropyLoss()
# Creamos un DataFrame de pandas para ir almacenando los valores calculados.
df = pd.DataFrame()
# Simulamos por tramos porque google colab se desconecta antes de que concluya para todos los valores de n en la lista.
min_valid_loss = 10000000.0
max_valid_accu = 0.0  
for k in range(num_k):
    # Creamos el modelo y el optimzador
    model = Net(n)
    # Dividimos el dataset de entrenamiento, el cual tiene 60000 muestras, en 60 partes de 1000 muestras.
    train_dataset,valid_dataset = random_split(train_dataset_orig,[50000,10000])
    # Creamos los dataloaders ...
    train_dataloader = DataLoader(train_dataset,batch_size=batch_size)
    valid_dataloader = DataLoader(valid_dataset,batch_size=batch_size)         
    #optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
    optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate,eps=1e-08,weight_decay=0,amsgrad=False)
    # Entrenamos el modelo y calcualmos curvas.
    min_valid_loss = float("inf")
    for epoch in range(num_epochs):
        train_loop(train_dataloader,model,loss_fn,optimizer)
        train_loss,train_accu = test_loop(train_dataloader,model,loss_fn)
        valid_loss,valid_accu = test_loop(valid_dataloader,model,loss_fn)
        print(f"n={n} k={k} epoch={epoch} train_loss={train_loss} train_accu={train_accu} valid_loss={valid_loss} valid_accu={valid_accu}")
        df = df.append({"n":n,
                        "k":k,
                        "epoch":epoch,
                        "train_loss":train_loss,
                        "train_accu":train_accu,
                        "valid_loss":valid_loss,
                        "valid_accu":valid_accu}
                        ,ignore_index=True)
        if min_valid_loss > valid_loss: # or max_valid_accu < valid_accu:
            if min_valid_loss > valid_loss:
                min_valid_loss = valid_loss
            if max_valid_accu < valid_accu:
                max_valid_accu = valid_accu
            # Guardamos los parámetros del modelo.
            model_fname = "best-model-"+init_datetime+".ptm"
            print("   Saving model_fname =",model_fname,end="")
            print(" ... DONE!")
            torch.save(model.state_dict(),model_fname)
json_fname = "simulation-results-"+init_datetime+".json"
df.to_json(json_fname)
if COLAB:
    files.download(model_fname)
    files.download(json_fname)

Resumen de simulaciones

    n=512 k=11 epoch=36 train_loss=0.17686420559883118 train_accu=0.93696 valid_loss=0.3091680943965912 valid_accu=0.8901
        Saving model_fname = best-model-2022-02-08-12-51-34.ptm ... DONE!
    best-model-2022-02-08-12-51-34.ptm

    simulation-results-2022-02-08-12-51-34.json

In [ ]:
%%bash --out list_json
# Usamos el bash magic de Jupyter para ver que archivos *.json hemos creado.
# Guardamos el resultado en la variable list_json
ls *.json

In [ ]:
list_json = list_json.split()
list_json

In [ ]:
df = pd.concat([pd.read_json(json_fname) for json_fname in list_json],ignore_index=True)
df

In [ ]:
df1 = df.drop("k",1)
df1

In [ ]:
df2 = df1.pivot_table(index=["n","epoch"],aggfunc="count").reset_index()
df2

In [ ]:
df3 = df1.pivot_table(index=["n","epoch"],aggfunc="mean").reset_index()
df3

In [ ]:
fig,axes=plt.subplots(1,2)
fig.set_size_inches(10.0,5.0)
colors = cm.Dark2.colors
for color,n in zip(colors,df["n"].unique()):
    dfn = df3[df3["n"]==n]
    x = dfn["epoch"]
    ax = axes[0]
    ax.set_xlabel("epoch")
    ax.set_ylabel("loss")
    ax.plot(x,dfn["train_loss"],label=f"train n={n}",color=color)
    ax.plot(x,dfn["valid_loss"],label=f"valid n={n}",color=color,linestyle='--')
    ax.legend()
    ax = axes[1]
    ax.set_xlabel("epoch")
    ax.set_ylabel("accuracy")
    ax.plot(x,dfn["train_accu"],label=f"train n={n}",color=color)
    ax.plot(x,dfn["valid_accu"],label=f"test n={n}",color=color,linestyle='--')
    ax.legend()
fig.tight_layout()
plt.show()

En este cómputo más detallado (con más estadística) del caso $n=512$, podemos ver que a partir de $\mathsf{epoch}\gtrsim 30$ el rendimiento de la red no mejora.

Probemos el último modelo guardado

    n=512 k=11 epoch=36 train_loss=0.17686420559883118 train_accu=0.93696 valid_loss=0.3091680943965912 valid_accu=0.8901
        Saving model_fname = best-model-2022-02-08-12-51-34.ptm ... DONE!

en los datos de prueba.

In [ ]:
%%bash --out model_fname
# Usamos el bash magic de Jupyter para ver que archivos *.json hemos creado.
# Guardamos el resultado en la variable list_json
ls *.ptm

In [ ]:
model_fname = model_fname.split()[0]

In [ ]:
n=512
model = Net(n)
model.load_state_dict(torch.load(model_fname,map_location="cpu"))
model.eval()
model = model.to(device)

In [ ]:
batch_size = 1000
loss_fn = nn.CrossEntropyLoss()
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=batch_size)
test_loss,test_accu = test_loop(test_loader,model,loss_fn)
print("test_loss = ",test_loss)
print("test_accu = ",test_accu)

Por comparación:

    epoch	n	train_accu	train_loss	valid_accu	valid_loss
    ...
    30	512	30	0.927252	0.202456	0.892600	0.303272
    31	512	31	0.928072	0.200221	0.892492	0.304235
    32	512	32	0.929298	0.197200	0.892708	0.304358
    33	512	33	0.930263	0.194121	0.892842	0.304515
    34	512	34	0.931963	0.190266	0.893242	0.303767
    35	512	35	0.932993	0.187327	0.893475	0.303990
    36	512	36	0.934033	0.184522	0.893075	0.304459
    37	512	37	0.935247	0.181606	0.893200	0.304804
    38	512	38	0.936440	0.178137	0.894133	0.304226
    39	512	39	0.937628	0.175377	0.893617	0.304836
    ...

Concluimos así que los valores de validación son confiables, y que $\mathsf{epoch} \approx 36$ constituye un adecuado valor de número de épocas de entrenamiento.